# Digital Slide Archive (DSA) Visualization Tutorial

Welcome to the Digial Slide Archive (DSA) visualization notebook! 

Digital Slide Archive (DSA) is a web application on which you can manage your pathology images and annotations. Luna provides a set of CLIs to help you convert your pathologist or model-generated annotations to a format that DSA accepts. For more details on the DSA platform, please refer to their [documentation](https://digitalslidearchive.github.io/digital_slide_archive/documentation/).

In this notebook, we will use **dsa** and **dsa_upload** CLIs to convert your annotations to a DSA compatible format and to upload them to DSA. We support results from Qupath/Stardist dectection models ([link to docker image](https://hub.docker.com/r/mskmind/qupath-stardist)), tile scores in a tabular format, and also expert annotations in geojson format. Here are the steps we will review:

- Setup DSA
- DSA Visuzaliation CLIs
- Upload Qupath regional annotation results
- Upload a heatmap generated from tile scores
- Upload bitmasks PNGs
- Upload bmp results
- Upload Stardist object detection results
- Upload Stardist cell detection results


In [1]:
!echo LUNA_HOME: $LUNA_HOME
!echo PYTHONPATH: $PYTHONPATH    

LUNA_HOME: /home/pashaa/vmount
PYTHONPATH:


## Setup DSA

Before running this notebook, make sure you have your pathology slides organized in DSA under a collection/folder. This will have already been done for you if you have run the previous notebooks in the tutorial.

![DSA Organization Screenshot](../img/dsa-organization-screenshot.png)

The collection name 'TCGA collection' and image file name 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs on DSA will be used while uploading the annotations.

## DSA Visualization CLIs

Luna Pathology offers 2 CLIs to help convert your annotation results to a DSA compatible json format and to upload them to DSA. The conversion and upload is divided in to 2 separate steps, so each step can be parallelized based on your computing capabilities and DSA platform setup. 

`dsa [COMMAND]` converts your annotation file format to a DSA compatible annotation json format.

`dsa_upload` uploads the DSA compatible annotation json.

Once upload is done, the annotations may be viewed in the HistomicsUI viewer.

*Note: Pushing and rendering a large number of annotation elements can take a long time. Please refer to DSA [documentation](https://digitalslidearchive.github.io/HistomicsTK/examples/tips_for_scalable_annotation_rendering), for user expectations and some tricks for managing annotations.*

In [48]:
# check available dsa cli commands
!dsa --help

2022-08-28 16:08:34,001 - INFO - root - Initalized logger, log file at: luna.log
Usage: dsa [OPTIONS] COMMAND [ARGS]...

  Convert segmentations, bitmasks, heatmaps to DSA annotation Json format.

Options:
  --help  Show this message and exit.

Commands:
  bitmask-polygon   Example:
  bmp-polygon       Example:
  heatmap           Example:
  qupath-polygon    Example:
  regional-polygon  Example:
  stardist-cell     Example:
  stardist-polygon  Example:


In [49]:
# check dsa cli commands help messages
!dsa stardist-polygon --help

2022-08-28 16:08:47,125 - INFO - root - Initalized logger, log file at: luna.log
Usage: dsa stardist-polygon [OPTIONS] INPUT

  Example:

          dsa stardist-polygon
              ../dsa_input/test_object_classification.geojson
              --output_dir ../dsa_annotations/stardist_polygon
              --annotation_name stardist_polygon_segmentations
              --image_filename 123.svs
              --line_colors '{"Other": "rgb(0,255,0)", "Lymphocyte": "rgb(255,0,0)"}'
              --fill_colors '{"Other": "rgba(0,255,0,100)", "Lymphocyte": "rgba(255,0,0,100)"}'

Options:
  -m, --method_param_path TEXT  path to a metadata json/yaml file with method
                                parameters to reproduce results
  -fc, --fill_colors TEXT       user-provided line color map with {feature
                                name:rgba values}
  -lc, --line_colors TEXT       user-provided line color map with {feature
                                name:rgb values}
  -a, --annotation_na

In [50]:
# check dsa_upload help messages
!dsa_upload --help

2022-08-28 16:08:53,545 - INFO - root - Initalized logger, log file at: luna.log
Usage: dsa_upload [OPTIONS] DSA_ENDPOINT

  DSA Annotation Upload CLI

  Example:     export DSA_USERNAME=username     export DSA_PASSWORD=password
  dsa_upload http://localhost:8080/dsa/api/v1         --collection_name tcga-
  data         --image_filename 123.svs         --annotation_filepath
  /path/to/dsa_annotation.json

Options:
  -c, --collection_name TEXT      name of the collection in DSA
  -f, --image_filename TEXT       name of the image file in DSA e.g. 123.svs
  -a, --annotation_filepath TEXT  path to a DSA annotation json file
  -u, --username TEXT             DSA username, can be inferred from
                                  DSA_USERNAME
  -p, --password TEXT             DSA password, should be inferred from
                                  DSA_PASSWORD
  -m, --method_param_path TEXT    path to a metadata json/yaml file with
                                  method parameters to reproduce

## Upload a heatmap generated from tile scores

the **heatmap** sub-command converts the output csv file containing tiles scores from the inference step to a DSA compatible json format. 

Once the json file is uploaded to DSA using the **upload** command, the heatmap may be viewed in HistomicsUI.

We use the color palette "viridis" where the output color ranges from purple to yellow, for scores from 0 to 1.

In [51]:
# example data_config
!dsa heatmap --help

2022-08-28 16:13:37,594 - INFO - root - Initalized logger, log file at: luna.log
Usage: dsa heatmap [OPTIONS] INPUT

  Example:

          dsa heatmap
              score.csv
              --output_dir ../dsa_annotations/heatmap
              --annotation_name heatmap
              --image_filename 123.svs
              --tile_size 256
              --column tumor
              --scale_factor 1

Options:
  -m, --method_param_path TEXT  path to a metadata json/yaml file with method
                                parameters to reproduce results
  -sc, --scale_factor INTEGER   scale to match image DSA. (default 1)
  -ts, --tile_size TEXT         tile size
  -c, --column TEXT             column to visualize e.g. tile_score
  -a, --annotation_name TEXT    name of the annotation to be displayed in DSA
  -f, --image_filename TEXT     name of the image file in DSA e.g. 123.svs
  -o, --output_dir TEXT         directory to save the DSA compatible
                                annotation json


In [52]:
# generate DSA compatible annotation
!dsa heatmap ../dsa_input/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.csv \
--output_dir ../dsa_annotations/heatmap \
--annotation_name heatmap \
--image_filename 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--tile_size 256 \
--column purple_score \
--scale_factor 1

# push annotation to DSA
!dsa_upload http://girder:8080/api/v1 \
--collection_name 'TCGA collection' \
--image_filename 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--annotation_filepath ../dsa_annotations/heatmap/purple_score_heatmap_01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.json \
--username admin \
--password password1

2022-08-28 16:14:13,154 - INFO - root - Initalized logger, log file at: luna.log
2022-08-28 16:14:13,159 - INFO - luna.common.utils - Started CLI Runner wtih <function heatmap_main at 0x7f0ac4dcdaf0>
2022-08-28 16:14:13,160 - INFO - luna.common.utils - Validating params...
2022-08-28 16:14:13,162 - INFO - luna.common.utils -  -> Set column (<class 'str'>) = purple_score
2022-08-28 16:14:13,163 - INFO - luna.common.utils -  -> Set tile_size (<class 'int'>) = 256
2022-08-28 16:14:13,164 - INFO - luna.common.utils -  -> Set annotation_name (<class 'str'>) = heatmap
2022-08-28 16:14:13,165 - INFO - luna.common.utils -  -> Set image_filename (<class 'str'>) = 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-08-28 16:14:13,167 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = ../dsa_annotations/heatmap
2022-08-28 16:14:13,169 - INFO - luna.common.utils -  -> Set input (<class 'str'>) = ../dsa_input/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.csv
2022-08-28 16:14:13,170 - INFO - lun

## Upload bitmasks PNGs

Simple PNG bitmasks can also be visualized in DSA. Use the **bitmask-polygon** sub-command and specify pngs with the corresponding labels. Then upload to view in HistomicsUI.

In [46]:
# generate DSA compatible annotation
!dsa bitmask-polygon \
'{"OTSU": "../PRO-12-123/tiling/inference/detect/otsu_mask.png"}' \
--output_dir ../dsa_annotations/bitmask \
--annotation_name bitmask \
--image_filename 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--line_colors '{"OTSU": "rgb(255,255,0)"}' \
--fill_colors '{"OTSU": "rgba(255,255,0,100)"}'

# push annotation to DSA
!dsa_upload http://girder:8080/api/v1 \
--collection_name 'TCGA collection' \
--image_filename 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--annotation_filepath ../dsa_annotations/bitmask/bitmask_01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.json \
--username admin \
--password password1

2022-08-27 11:40:35,648 - INFO - root - Initalized logger, log file at: luna.log
2022-08-27 11:40:35,652 - INFO - luna.common.utils - Started CLI Runner wtih <function bitmask_polygon_main at 0x7f789e1eaa60>
2022-08-27 11:40:35,653 - INFO - luna.common.utils - Validating params...
2022-08-27 11:40:35,655 - INFO - luna.common.utils -  -> Set fill_colors (<class 'dict'>) = {'OTSU': 'rgba(255,255,0,100)'}
2022-08-27 11:40:35,656 - INFO - luna.common.utils -  -> Set line_colors (<class 'dict'>) = {'OTSU': 'rgb(255,255,0)'}
2022-08-27 11:40:35,657 - INFO - luna.common.utils -  -> Set annotation_name (<class 'str'>) = bitmask
2022-08-27 11:40:35,658 - INFO - luna.common.utils -  -> Set image_filename (<class 'str'>) = 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-08-27 11:40:35,659 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = ../dsa_annotations/bitmask
2022-08-27 11:40:35,661 - INFO - luna.common.utils -  -> Set input (<class 'dict'>) = {'OTSU': '../PRO-12-123/tiling/i

## Upload bmp results

Similary, BMP is another file format that may be used to store segmentation or classification results. **bmp-polygon** converts the bmps to the DSA compatible json annotation.

In [53]:
!dsa bmp-polygon --help

2022-08-28 16:17:53,564 - INFO - root - Initalized logger, log file at: luna.log
Usage: dsa bmp-polygon [OPTIONS] INPUT

  Example:

          dsa bmp-polygon
              results.bmp
              --output_dir ../dsa_annotations/bmp
              --annotation_name bmp
              --image_filename 123.svs
              --label '{0: "Tumor", 1: "Other"}'
              --scale_factor 1
              --line_colors '{"Other": "rgb(0,255,0)", "Tumor": "rgb(255,0,0)"}'
              --fill_colors '{"Other": "rgba(0,255,0,100)", "Tumor": "rgba(255,0,0,100)"}'

Options:
  -m, --method_param_path TEXT  path to a metadata json/yaml file with method
                                parameters to reproduce results
  -sc, --scale_factor TEXT      scale to match image DSA. (default 1)
  -fc, --fill_colors TEXT       user-provided line color map with {feature
                                name:rgba values}
  -lc, --line_colors TEXT       user-provided line color map with {feature
                

## Upload Qupath regional annotation results

Regional annotations generated by [Qupath](https://qupath.github.io/) includes regional polygons from object detection along with nuclear properties.

For object and cell detection models in QuPath, please checkout our [Qupath/Stardist docker](https://hub.docker.com/r/mskmind/qupath-stardist).

In [54]:
# generate DSA compatible annotation
!dsa qupath-polygon \
../PRO-12-123/data/toy_data_set/table/ANNOTATIONS/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.annotation.geojson \
--output_dir ../dsa_annotations/quppath \
--annotation_name quppath \
--image_filename 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--classes_to_include Other \
--line_colors '{"Other": "rgb(0,255,255)"}' \
--fill_colors '{"Other": "rgba(0,255,255,100)"}'

2022-08-28 16:21:26,204 - INFO - root - Initalized logger, log file at: luna.log
2022-08-28 16:21:26,207 - INFO - luna.common.utils - Started CLI Runner wtih <function qupath_polygon_main at 0x7fb040f5e9d0>
2022-08-28 16:21:26,209 - INFO - luna.common.utils - Validating params...
2022-08-28 16:21:26,210 - INFO - luna.common.utils -  -> Set fill_colors (<class 'dict'>) = {'Other': 'rgba(0,255,255,100)'}
2022-08-28 16:21:26,211 - INFO - luna.common.utils -  -> Set line_colors (<class 'dict'>) = {'Other': 'rgb(0,255,255)'}
2022-08-28 16:21:26,213 - INFO - luna.common.utils -  -> Set annotation_name (<class 'str'>) = quppath
2022-08-28 16:21:26,214 - INFO - luna.common.utils -  -> Set image_filename (<class 'str'>) = 01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-08-28 16:21:26,215 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = ../dsa_annotations/quppath
2022-08-28 16:21:26,216 - INFO - luna.common.utils -  -> Set input (<class 'str'>) = ../PRO-12-123/data/toy_data_set/

In [4]:
!ls -l  /home/pashaa/.local/bin/

total 928
drwxr-xr-x 2 pashaa pashaa 4096 Aug  8 22:22 __pycache__
-rwxr-xr-x 1 pashaa pashaa  229 Aug  8 22:22 alt-java
-rwxr-xr-x 1 pashaa pashaa  229 Aug  8 22:22 appletviewer
-rwxr-xr-x 1 pashaa pashaa  220 Aug  8 22:22 applygeo
-rwxr-xr-x 1 pashaa pashaa  219 Aug  8 22:24 bokeh
-rwxr-xr-x 1 pashaa pashaa  221 Aug  8 22:22 cct
-rwxr-xr-x 1 pashaa pashaa  226 Aug  8 22:24 chkfilehash
-rwxr-xr-x 1 pashaa pashaa  221 Aug  8 22:22 cjpeg
-rwxr-xr-x 1 pashaa pashaa  229 Aug  8 22:22 clhsdb
-rwxr-xr-x 1 pashaa pashaa  258 Aug  8 22:31 convert-caffe2-to-onnx
-rwxr-xr-x 1 pashaa pashaa  258 Aug  8 22:31 convert-onnx-to-caffe2
-rwxr-xr-x 1 pashaa pashaa  221 Aug  8 22:22 cs2cs
-rwxr-xr-x 1 pashaa pashaa  235 Aug  8 22:22 dask-scheduler
-rwxr-xr-x 1 pashaa pashaa  229 Aug  8 22:22 dask-ssh
-rwxr-xr-x 1 pashaa pashaa  232 Aug  8 22:22 dask-worker
-rwxr-xr-x 1 pashaa pashaa  982 Aug  8 22:32 detect_tissue
-rwxr-xr-x 1 pashaa pashaa  216 Aug  8 22:24 dirhash
-rwxr-xr-x 1 pashaa pashaa  221 Aug  

In [45]:
# push annotation to DSA
!dsa_upload http://localhost:8080/dsa/api/v1 \
--collection_name tcga-data \
--image_filename 123.svs \
--annotation_filepath ../dsa_annotations/qupath/quppath_123.json

Successfully connected to DSA.
collection_id_dict {'_accessLevel': 2, '_id': '60807410150bd39c9df579cf', '_modelType': 'collection', '_textScore': 15.75, 'created': '2021-04-21T18:50:56.730000+00:00', 'description': 'test images', 'meta': {}, 'name': 'test-path', 'public': True, 'publicFlags': [], 'size': 2350018806, 'updated': '2021-04-21T18:50:56.730000+00:00'}
Collection test-path found with id: 60807410150bd39c9df579cf
Annotation successfully pushed to DSA.
Time to push annotation 0.0221097469329834
http://localhost:8080/histomics#?image=60807ad8150bd39c9df579d2


## Upload Stardist object detection results

Stardist is a nuclear segmentation algorithm that is quite capable in detecting and segmenting cells/nuclei in pathology images. **stardist-polygon** option converts Stardist object detection results as polygons capturing different types of cells.

*Note: this command can take a few minutes if object detection is run on the whole slide.*

In [16]:
# generate DSA annotation
!dsa stardist-polygon \
../dsa_input/test_object_classification.geojson \
--output_dir ../dsa_annotations/stardist_polygon \
--annotation_name stardist_polygon_segmentations \
--image_filename 123.svs \
--line_colors '{"Other": "rgb(0,255,0)", "Lymphocyte": "rgb(255,0,0)"}' \
--fill_colors '{"Other": "rgba(0,255,0,100)", "Lymphocyte": "rgba(255,0,0,100)"}'

2022-04-05 09:06:50,899 - INFO - root - FYI: Initalized logger, log file at: data-processing.log with handlers: [<StreamHandler <stderr> (INFO)>, <RotatingFileHandler /Users/rosed2/Documents/msk-mind/luna/docker/luna_tutorial/vmount/notebooks/data-processing.log (INFO)>]
2022-04-05 09:06:50,900 - INFO - luna.common.utils - Running <function stardist_polygon_main at 0x133e7f280> with {'output_dir': '../dsa_annotations/stardist_polygon', 'annotation_name': 'stardist_polygon_segmentations', 'image_filename': '123.svs', 'line_colors': '{"Other": "rgb(0,255,0)", "Lymphocyte": "rgb(255,0,0)"}', 'fill_colors': '{"Other": "rgba(0,255,0,100)", "Lymphocyte": "rgba(255,0,0,100)"}', 'input': '../dsa_input/test_object_classification.geojson', 'method_param_path': None}
2022-04-05 09:06:50,900 - INFO - luna.common.utils - Param fill_colors set = {'Other': 'rgba(0,255,0,100)', 'Lymphocyte': 'rgba(255,0,0,100)'}
2022-04-05 09:06:50,900 - INFO - luna.common.utils - Param line_colors set = {'Other': 'rg

In [20]:
# check json annotation
!head ../dsa_annotations/stardist_polygon/stardist_polygon_segmentations_123.json

{"description": "", "elements": [{"fillColor": "rgba(0,255,0,100)", "lineColor": "rgb(0,255,0)", "lineWidth": 2, "type": "polyline", "closed": true, "points": [[61781.95, 929.31, 0], [61771.77, 932.48, 0], [61763.49, 938.38, 0], [61757.63, 946.76, 0], [61756.68, 949.76, 0], [61756.34, 957.75, 0], [61756.81, 960.34, 0], [61759.75, 967.25, 0], [61761.4, 969.48, 0], [61766.69, 973.96, 0], [61774.61, 976.99, 0], [61776.99, 977.25, 0], [61787.6, 976.28, 0], [61793.96, 974.36, 0], [61803.92, 972.54, 0], [61812.41, 969.1, 0], [61819.84, 962.2, 0], [61821.52, 958.67, 0], [61822.29, 950.5, 0], [61821.42, 947.99, 0], [61816.19, 940.43, 0], [61810.58, 936.37, 0], [61803.19, 933.06, 0], [61793.31, 929.64, 0], [61781.95, 929.31, 0]], "label": {"value": "Other"}}], "name": "stardist_polygon_segmentations"}

In [22]:
# push annotation to DSA
!dsa_upload http://localhost:8080/dsa/api/v1 \
--collection_name tcga-data \
--image_filename 123.svs \
--annotation_filepath ../dsa_annotations/stardist_polygon/stardist_polygon_segmentations_123.json

env: DSA_USERNAME=username
env: DSA_PASSWORD=password
Traceback (most recent call last):
  File "/Users/rosed2/Documents/msk-mind/env/bin/dsa_upload", line 33, in <module>
    sys.exit(load_entry_point('pyluna-pathology==0.1.1', 'console_scripts', 'dsa_upload')())
  File "/Users/rosed2/Documents/msk-mind/env/bin/dsa_upload", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/usr/local/Cellar/python@3.9/3.9.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/importlib/metadata.py", line 77, in load
    module = import_module(match.group('module'))
  File "/usr/local/Cellar/python@3.9/3.9.9/Frameworks/Python.framework/Versions/3.9/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_lo

This is a screenshot form HistomicsUI, the high-magnification viewer. You can zoom in and view your annotation results with the desired opacity. As specified in `dsa_configs/stardist_polygon_config.yaml`, the red objects are classified as lymphocytes and the green cells are "other" cells.

![Stardist Polygon Screenshot](../img/stardist-polygon-screenshot.png)


## Upload Stardist cell detection results

Here we use cellular detection results generated from Stardist. The x,y coordinates of the cells in the input TSV file will be visualized as a point, as opposed to a more complex polygon that we saw in the previous step with **stardist-polygon**. You'll notice that the point annotation is faster to upload compared to the polygon represenation of the cells.

We also set fill color alpha value to 0 makes annotation upload faster.

In [26]:
# generate DSA annotation
!dsa stardist-cell \
../dsa_input/test_object_detection.tsv \
--output_dir ../dsa_annotations/stardist_cell \
--annotation_name stardist_cell_segmentations \
--image_filename 123.svs \
--line_colors '{"Other": "rgb(0,255,0)", "Lymphocyte": "rgb(255,0,0)"}' \
--fill_colors '{"Other": "rgba(0,255,0,100)", "Lymphocyte": "rgba(255,0,0,100)"}'

2022-04-05 09:19:48,224 - INFO - root - FYI: Initalized logger, log file at: data-processing.log with handlers: [<StreamHandler <stderr> (INFO)>, <RotatingFileHandler /Users/rosed2/Documents/msk-mind/luna/docker/luna_tutorial/vmount/notebooks/data-processing.log (INFO)>]
2022-04-05 09:19:48,225 - INFO - luna.common.utils - Running <function stardist_cell_main at 0x13c902550> with {'output_dir': '../dsa_annotations/stardist_cell', 'annotation_name': 'stardist_cell_segmentations', 'image_filename': '123.svs', 'line_colors': '{"Other": "rgb(0,255,0)", "Lymphocyte": "rgb(255,0,0)"}', 'fill_colors': '{"Other": "rgba(0,255,0,100)", "Lymphocyte": "rgba(255,0,0,100)"}', 'input': '../dsa_input/test_object_detection.tsv', 'method_param_path': None}
2022-04-05 09:19:48,225 - INFO - luna.common.utils - Param fill_colors set = {'Other': 'rgba(0,255,0,100)', 'Lymphocyte': 'rgba(255,0,0,100)'}
2022-04-05 09:19:48,225 - INFO - luna.common.utils - Param line_colors set = {'Other': 'rgb(0,255,0)', 'Lymp

In [27]:
# check json annotation
!head ../dsa_annotations/stardist_cell/stardist_cell_segmentations_123.json

{"description": "", "elements": [{"fillColor": "rgba(0,255,0,100)", "lineColor": "rgb(0,255,0)", "lineWidth": 2, "type": "point", "center": [61788.06625424067, 953.4224705647575, 0], "label": {"value": "Other"}}, {"fillColor": "rgba(0,255,0,100)", "lineColor": "rgb(0,255,0)", "lineWidth": 2, "type": "point", "center": [63033.92536419876, 957.214128916384, 0], "label": {"value": "Other"}}, {"fillColor": "rgba(0,255,0,100)", "lineColor": "rgb(0,255,0)", "lineWidth": 2, "type": "point", "center": [64150.069846338054, 957.3538215924966, 0], "label": {"value": "Other"}}], "name": "stardist_cell_segmentations"}

In [29]:
# push annotation to DSA
!dsa_upload http://localhost:8080/dsa/api/v1 \
--collection_name tcga-data \
--image_filename 123.svs \
--annotation_filepath ../dsa_annotations/stardist_cell/stardist_cell_segmentations_123.json

Successfully connected to DSA.
collection_id_dict {'_accessLevel': 2, '_id': '60807410150bd39c9df579cf', '_modelType': 'collection', '_textScore': 15.75, 'created': '2021-04-21T18:50:56.730000+00:00', 'description': 'test images', 'meta': {}, 'name': 'test-path', 'public': True, 'publicFlags': [], 'size': 936444465, 'updated': '2021-04-21T18:50:56.730000+00:00'}
Collection test-path found with id: 60807410150bd39c9df579cf
Annotation successfully pushed to DSA.
Time to push annotation 202.15096521377563
http://localhost:8080/histomics#?image=60807ad8150bd39c9df579d2


Below is another screenshot from HistomicsUI, from the link printed above. The results are the same as **stardist-polygon** visualization. Notice the cells are captured more minimally as circles, and not polygons. For rapid prototyping, **stardist-cell** offers faster annotation upload speed compared to **stardist-polygon**.

![Stardist Cell Screenshot](../img/stardist-cell-screenshot.png)

Congratulations! Now you can visualize your annotations and results on DSA platform.